In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [2]:
# Read data set
datasetPath = Path('specgrams')

losslessData = {}
lossyData = {}

filenameList = []

# Read the lossless data
for childPath in datasetPath.joinpath('specgram_lossless').iterdir():
    if childPath.suffix == '.tiff':
        filename = childPath.name.replace('.tiff', '')
        filenameList.append(filename)
        # Flatten to 1D array in order to use as feature
        losslessData[filename] = plt.imread(childPath.as_posix()).flatten()
        plt.close('all')
print('Lossless data loaded: ', len(losslessData))
        
# Read the lossy data
for childPath in datasetPath.joinpath('specgram_lossy_insanely_high').iterdir():
    if childPath.suffix == '.tiff':
        filename = childPath.name.replace('.mp3', '').replace('.tiff', '')
        # This should not happen, but we should check if the lossy image file
        # corresponds to a lossless image file
        if not filename in losslessData:
            print('WARNING: Mismatched lossy file: ', filename)
            continue
        # Flatten to 1D array in order to use as feature
        lossyData[filename] = plt.imread(childPath.as_posix()).flatten()
        plt.close('all')
print('Lossy data loaded: ', len(losslessData))

Lossless data loaded:  6877
Lossy data loaded:  6877


In [3]:
# Naive approach to split training/testing set
# Run either this cell or the next cell, which uses sklearn's split function
trainTestRatio = 0.8

# Label: lossless = 0, lossy = 1
trainingLabelList = []
trainingSetList = []
testingLabelList = []
testingSetList = []

for name in losslessData:
    # Skip if the track is only in lossless data
    if not name in lossyData:
        continue
    # Add the entry to training set if the training set is not big enough
    if len(trainingSetList) < (2 * len(lossyData)) * trainTestRatio:
        # Add both lossless entry and the lossy counterpart
        trainingSetList.append(losslessData[name])
        trainingLabelList.append(0)
        trainingSetList.append(lossyData[name])
        trainingLabelList.append(1)
    else:
        testingSetList.append(losslessData[name])
        testingLabelList.append(0)
        testingSetList.append(lossyData[name])
        testingLabelList.append(1)
print('Training set: ', len(trainingSetList))
print('Testing set: ', len(testingSetList))

Training set:  11004
Testing set:  2750


In [3]:
# Split train and test set using sklearn's API
from sklearn.model_selection import train_test_split
X = list(losslessData.values()) + list(lossyData.values())
y = [0 for _ in losslessData] + [1 for _ in lossyData]
trainingSetList, testingSetList, trainingLabelList, testingLabelList = train_test_split(X, y, test_size=0.2)

In [4]:
# Initialize decision tree classifier
from sklearn import tree
clf = tree.DecisionTreeClassifier()

In [10]:
# Fit and test classifier
clf.fit(trainingSetList, trainingLabelList)
clf.score(testingSetList, testingLabelList)

0.9825517993456925

In [5]:
# Let SKlearn cross-validate classifier
scoring = ['f1', 'precision', 'recall', 'accuracy']
from sklearn.model_selection import cross_validate
results = cross_validate(clf, X, y, cv=5, scoring=scoring)

In [6]:
results

{'fit_time': array([3006.15935135, 2303.35388303, 2898.73660469, 2529.66961598,
        2831.16696763]),
 'score_time': array([0.9221859 , 1.30297637, 0.9370389 , 1.29531193, 1.3944962 ]),
 'test_f1': array([0.95851528, 0.95845481, 0.95560408, 0.95639535, 0.96261343]),
 'test_precision': array([0.9592134 , 0.96055515, 0.95699708, 0.95639535, 0.96086957]),
 'test_recall': array([0.95781818, 0.95636364, 0.95421512, 0.95639535, 0.96436364]),
 'test_accuracy': array([0.95856052, 0.95856052, 0.95565249, 0.9563795 , 0.96254545])}

In [8]:
print('f1: ', np.mean(results['test_f1']))
print('precision: ', np.mean(results['test_precision']))
print('recall: ', np.mean(results['test_recall']))
print('accuracy: ', np.mean(results['test_accuracy']))

f1:  0.9583165897988171
precision:  0.9588061099316085
recall:  0.9578311839323467
accuracy:  0.958339697961072
